In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
#from createDataset.params import *
import os
import json
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
LOCAL_PATH = "../data/"
WIKI_DATA = os.path.join(LOCAL_PATH, "wikipedia_male_female.json")

In [ ]:
# create the session
spark = SparkSession.builder.getOrCreate()

# create the context
sc = spark.sparkContext

# Load data frame
df = spark.read.json(WIKI_DATA)
df.show()

In [ ]:
# Open Gender dictionary
with open('../data/dict_genders.json') as json_file:
    line = json_file.readline()
    dict_genders = json.loads(line)
    
# Open occupations dictionary
dict_occupations = {}
with open('../data/dict_occupations.json') as json_file:
    content = json_file.readlines()
    for line in content:
        occ = json.loads(line)
        dict_occupations.update(occ)
        
# Observation: We need dict_categories_occupations.json in the data folder
# Open occupations categories dictionary
with open('../data/dict_categories_occupations.json') as json_file:
    line = json_file.readline()
    dict_cat_occ = json.loads(line)

# Create function to translate a code into a category
def translate(mapping):
    def translate_(col):
        return mapping.get(col)
    return udf(translate_, StringType())

In [ ]:
# Translate gender and occupations codes into corresponding labels
df = df.withColumn('gender', translate(dict_genders)('gender'))\
       .withColumn('occupation', explode(split(regexp_replace(regexp_replace(regexp_replace\
                                (regexp_replace(df['occupation'], '\\[', ''), '\\]', ''), ' ', ''),"'", ""), ",")))\
       .filter(col('occupation') != '')\
       .withColumn('occupation', translate(dict_occupations)('occupation'))\
       .withColumn('field', translate(dict_cat_occ)('occupation'))

df.show()

In [ ]:
# Query to know how many males and females are in the data frame
# Observation: When occupation translation is done, the observations without a label are dropped, that's why, there are less male and female
df.registerTempTable("df")

query = """
SELECT gender, count(DISTINCT id) as count
FROM df
GROUP BY gender
ORDER BY count DESC
"""

gender_counts = spark.sql(query)
gender_counts = gender_counts.toPandas()
gender_counts

In [ ]:
pl = gender_counts.plot(kind="bar", x="gender", y="count", figsize=(10, 7), log=True, alpha=0.5, color="green")
pl.set_xlabel("Gender")
pl.set_ylabel("Number of biographies (Log scale)")
pl.set_title("Number of biographies by gender");

In [ ]:
df.registerTempTable("df")

query = """
SELECT field, count(DISTINCT id) as count
FROM df
WHERE field IS NOT NULL
GROUP BY field
ORDER BY count DESC
"""

occu_cat_counts = spark.sql(query)
occu_cat_counts = occu_cat_counts.toPandas()
occu_cat_counts

In [ ]:
pl = occu_cat_counts.plot(kind="bar", x="field", y="count", figsize=(10, 7), log=False, alpha=0.5, color="green")
pl.set_xlabel("Field of occupation")
pl.set_ylabel("Number of biographies")
pl.set_title("Number of biographies by field of occupation");

In [ ]:
n_total = occu_cat_counts['count'].sum()
n_artist = occu_cat_counts[occu_cat_counts['field'] == 'Artist']['count'].values[0]
n_sports = occu_cat_counts[occu_cat_counts['field'] == 'Sports']['count'].values[0]
n_politics = occu_cat_counts[occu_cat_counts['field'] == 'Politics']['count'].values[0]

print("{:.2f}% of the entries work in the artistic field".format(n_artist/n_total*100))
print("{:.2f} % of the entries work in the sports field".format(n_sports/n_total*100))
print("{:.2f} % of the entries work in the politics field".format(n_politics/n_total*100))

In [ ]:
df.registerTempTable("df")

query = """
SELECT field, gender, count(DISTINCT id) as count
FROM df
WHERE field IS NOT NULL
GROUP BY field, gender
ORDER BY field, gender
"""

occu_gender_counts = spark.sql(query)
occu_gender_counts = occu_gender_counts.toPandas()
occu_gender_counts

In [ ]:
male_count = occu_gender_counts[occu_gender_counts['gender'] == 'male']['count'].tolist()
female_count = occu_gender_counts[occu_gender_counts['gender'] == 'female']['count'].tolist()
index = occu_gender_counts['field'].unique().tolist()
occ_by_gender = pd.DataFrame({'male': male_count, 'female': female_count}, index=index)

pl = occ_by_gender.plot(kind="bar", figsize=(10, 7), log=False, alpha=0.5, color=["green", "red"])
pl.set_xlabel("Field of occupation")
pl.set_ylabel("Number of biographies")
pl.set_title("Number of biographies by gender and field of occupation");